<a href="https://colab.research.google.com/github/Georgemburu/MACHINE-LEARNING/blob/master/Text_Generation_Revisited_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x


TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, print_function, unicode_literals,division

import tensorflow as tf
import numpy as np
import os 
import time

In [3]:
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt',
    origin='https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)

1122304/1115394 [==============================] - 0s 0us/step


In [4]:
# Read the data
text = open(path_to_file,'rb').read().decode(encoding='utf-8')
print('length of text:',len(text))

length of text: 1115394


In [6]:
# Take a look at some text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
# Creat a vocab
vocab =sorted(set(text))
print('Vocab length:',len(vocab))

Vocab length: 65


In [0]:
# Process The Text
# 1. Vectorize the text
char2idx = {c:i for i,c in enumerate(vocab)}
idx2char = {i:c for i,c in enumerate(vocab)}

text_as_int = np.array([char2idx[c] for c in text])

In [11]:
text_as_int[:250]

array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
       44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39,
       52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1,
       51, 43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31,
       54, 43, 39, 49,  6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56,
       57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39,
       56, 43,  1, 39, 50, 50,  1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56,
       39, 58, 46, 43, 56,  1, 58, 53,  1, 42, 47, 43,  1, 58, 46, 39, 52,
        1, 58, 53,  1, 44, 39, 51, 47, 57, 46, 12,  0,  0, 13, 50, 50, 10,
        0, 30, 43, 57, 53, 50, 60, 43, 42,  8,  1, 56, 43, 57, 53, 50, 60,
       43, 42,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43,
       52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63, 53, 59,  1, 49, 52, 53,
       61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41, 47, 59, 57,  1, 47,
       57,  1, 41, 46, 47

In [0]:
# function to decode text tensor
def decode_sentence(sentence):
  return ''.join(idx2char[c] for c in sentence)

In [0]:
# function to encode sentence
def encode_sentence(sentence):
  return np.array([char2idx[c] for c in sentence])

In [18]:
decode_sentence(text_as_int[:250])

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n'

In [23]:
encode_sentence(decode_sentence(text_as_int[:250]))== text_as_int[:250]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [24]:
# PREDICTION TASK
seq_length = 100
examples_per_epochs = len(text)//(seq_length+1)
# create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [34]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(decode_sentence(item.numpy()))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki
ll him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be d
one: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citi


In [0]:
# Split functions 
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [38]:
for inp, targ in dataset.take(1):
  print('Inp:->',inp)
  print('Targ:->',targ)

Inp:-> tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int64)
Targ:-> tf.Tensor(
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1], shape=(100,), dtype=int64)


In [39]:
# Create training Batches
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# BUILD THE MODEL
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size,embedding_dim,rnn_units,batch_size):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Embedding(vocab_size,embedding_dim,
                                batch_input_shape=[batch_size,None]),
      tf.keras.layers.GRU(rnn_units,
                          return_sequences=True,
                          stateful=True,
                          recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)


In [43]:
# Test the model
for input_example_batch, targ_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions)
  print(example_batch_predictions.shape)
  print('batch_size, sequence_length,vocab_size')

tf.Tensor(
[[[ 1.94497034e-02 -9.27703269e-03 -1.53801795e-02 ... -6.97805081e-04
   -8.65260884e-03 -1.17516657e-02]
  [ 8.53184145e-03 -5.49715152e-03 -1.09994942e-02 ... -6.37609977e-04
    1.05899666e-03 -1.44618601e-02]
  [ 2.51820032e-03  6.09260052e-04 -6.12176256e-04 ... -3.81841278e-03
    4.13145218e-03 -1.68056376e-02]
  ...
  [-2.05953214e-02 -3.73741984e-03  1.95849333e-02 ... -1.54804438e-05
    8.80505331e-03 -4.41480055e-03]
  [-6.89258240e-03 -2.89372448e-03  1.63163710e-02 ...  1.80321536e-03
    1.54160783e-02 -1.10499114e-02]
  [-8.89282115e-03  1.82306720e-03  1.25508392e-02 ... -3.46512953e-03
    1.22613534e-02 -1.71570070e-02]]

 [[ 6.42162003e-03  3.61999497e-03  1.44417379e-02 ...  8.24582018e-03
    8.74631666e-03  1.27377445e-02]
  [ 6.95883855e-03  1.59812439e-02  1.79161653e-02 ...  6.57138368e-03
    1.60858706e-02  2.18497007e-04]
  [ 4.26428858e-04  6.16571121e-03  1.49651598e-02 ...  5.12869284e-03
    2.32984917e-03  6.96100155e-03]
  ...
  [ 1.190292

In [49]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 65), dtype=float32, numpy=
array([[ 1.94497034e-02, -9.27703269e-03, -1.53801795e-02, ...,
        -6.97805081e-04, -8.65260884e-03, -1.17516657e-02],
       [ 8.53184145e-03, -5.49715152e-03, -1.09994942e-02, ...,
        -6.37609977e-04,  1.05899666e-03, -1.44618601e-02],
       [ 2.51820032e-03,  6.09260052e-04, -6.12176256e-04, ...,
        -3.81841278e-03,  4.13145218e-03, -1.68056376e-02],
       ...,
       [-2.05953214e-02, -3.73741984e-03,  1.95849333e-02, ...,
        -1.54804438e-05,  8.80505331e-03, -4.41480055e-03],
       [-6.89258240e-03, -2.89372448e-03,  1.63163710e-02, ...,
         1.80321536e-03,  1.54160783e-02, -1.10499114e-02],
       [-8.89282115e-03,  1.82306720e-03,  1.25508392e-02, ...,
        -3.46512953e-03,  1.22613534e-02, -1.71570070e-02]], dtype=float32)>

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [48]:
sampled_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([ 3, 47, 37, 64, 57, 51, 48, 20, 37, 63, 42, 14, 34, 51, 23, 64, 44,
       31, 35,  4, 49, 59, 45, 41, 12, 40, 57, 25, 33, 50, 49, 33, 32,  3,
        3,  8, 16,  9, 11, 30, 64, 23, 57, 61, 27, 35, 57, 27, 48, 59, 22,
       20, 19, 56, 52, 52, 45, 63, 35, 58, 25, 51, 11, 53, 14, 23, 31, 38,
       14, 44, 37, 13, 35,  8, 51,  5, 47, 60,  5,  7, 35, 14, 48, 59,  5,
       27, 50, 25, 38, 26, 46, 64, 43, 15, 55, 11, 50, 17, 44, 61])

In [52]:
sampled_indices.shape

(100,)

In [50]:
# decode those to see the prediction from untrained model
decode_sentence(sampled_indices)

"$iYzsmjHYydBVmKzfSW&kugc?bsMUlkUT$$.D3;RzKswOWsOjuJHGrnngyWtMm;oBKSZBfYAW.m'iv'-WBju'OlMZNhzeCq;lEfw"

In [53]:
# Train the model
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

example_batch_loss = loss(targ_example_batch, example_batch_predictions)
print('Pediction shape: ',example_batch_loss.shape)
print('scalar_loss:  ', example_batch_loss.numpy().mean())

Pediction shape:  (64, 100)
scalar_loss:   4.173311


In [0]:
# compile the model
model.compile(optimizer='adam',loss=loss)

In [0]:
# Configure checkpoints
checkpoints_dir = './trainig_checkpoints'
checkpoint_prefix = os.path.join(checkpoints_dir,'chkpt_{epoch}')

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [58]:
# Execute the training
EPOCHS = 10
history = model.fit(dataset,epochs=EPOCHS,callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/10
172/172 [==============================] - 13s 77ms/step - loss: 2.7166
Epoch 2/10
172/172 [==============================] - 12s 70ms/step - loss: 1.9772
Epoch 3/10
172/172 [==============================] - 12s 69ms/step - loss: 1.7040
Epoch 4/10
172/172 [==============================] - 12s 68ms/step - loss: 1.5522
Epoch 5/10
172/172 [==============================] - 12s 69ms/step - loss: 1.4612
Epoch 6/10
172/172 [==============================] - 12s 68ms/step - loss: 1.4004
Epoch 7/10
172/172 [==============================] - 12s 69ms/step - loss: 1.3541
Epoch 8/10
172/172 [==============================] - 12s 70ms/step - loss: 1.3138
Epoch 9/10
172/172 [==============================] - 12s 69ms/step - loss: 1.2797
Epoch 10/10
172/172 [==============================] - 12s 69ms/step - loss: 1.2467


In [63]:
# Restore the latest checkpoint
# GENERATE TEXT
model = build_model(
    vocab_size,
    embedding_dim,
    rnn_units,
    batch_size=1
)
model.load_weights(tf.train.latest_checkpoint(checkpoints_dir))
model.build(tf.TensorShape([1,None]))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_5 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_5 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model,start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval,0)

  text_generated = []
  temperature = 1.0
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimention
    predictions = tf.squeeze(predictions,0)

    predictions = predictions/temperature
    predicted_id = tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id],0)
    text_generated.append(idx2char[predicted_id])
  return (start_string +''.join(text_generated))


In [69]:
print(generate_text(model, start_string=u"ROMEO: "))


ROMEO: God s,
As it is war have made for the honour of his phording Richard Scend,
Aumored over my life for leave to sctre.

TRANIO:
Hair for his form, eyes.
Grace as fair sorrow, before me love? thou
art a guilt; his purpose may be short betimes
Upon his body, in one three-like gentleman in
this scletence as those knaves as thoughts to praised, I do command.

PETRUCHIO:
It is a maid words, Tappine and quicklawam, and none;
Let them I make you falling to keep agree.

First Gentleman:
Shall I see this shophers; would it is ready to him.
Nor now, good man! let's apoll and Dar Helves share, he had left a
Nave hamina, in company, carrilming bleeds with nail:
We came utouche you to hair another's traungoon, and like itself,
Who had be rathers cilled up the terms of her,
Bloody treed.

Second Citizen:
You do, my son! O mercy more, no more! I say I see the crown.

KING HENRY VI

WARWICK:

TRANIO:
Master, my soul
Bole complexerace.

BIANCA:
Was ever medown against the malmanded arms.'

FRIAR T